In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import Convex as cvx 
import MeshCat as mc 
using Test
using Random
import Convex as cvx 
import ForwardDiff as FD
import ECOS      
include(joinpath(@__DIR__,"ref_traj.jl"))
include(joinpath(@__DIR__,"planar_hexrotor_dynamics.jl"))

In [ ]:
function kalman_filter(A, B, C, D, Q, R, x_hat, P, u, y, W, w)
    # w is the wind input 
    # W is the system response to wind 
    x_hat_minus = A * x_hat + B * u + W * w
    P_minus = A * P * A' + Q
    K = P_minus * C' / (C * P_minus * C' + R)
    x_hat_plus = x_hat_minus + K * (y - C * x_hat_minus - D * u)
    P_plus = (I - K * C) * P_minus
    
    return x_hat_plus, P_plus
end

function update_wind(current_wind, dt, intensity)
    # Random walk update
    d_wind = randn(size(current_wind)) .* sqrt(intensity * dt)
    new_wind = current_wind + d_wind
    return new_wind
end



In [ ]:
"""
`u = convex_mpc(A,B,X_ref_window,xic,xg,u_min,u_max,N_mpc)`

setup and solve the above optimization problem, returning the 
first control u_1 from the solution (should be a length nu 
Vector{Float64}).  
"""
function convex_mpc(A::Matrix, # discrete dynamics matrix A
                    B::Matrix, # discrete dynamics matrix B
                    X_ref_window::Vector{Vector{Float64}}, # reference trajectory for this window 
                    xic::Vector, # current state x 
                    u_min::Vector, # lower bound on u 
                    u_max::Vector, # upper bound on u 
                    N_mpc::Int64,  # length of MPC window (horizon)
                    Q, #Q cost
                    R, #Input cost
                    wind_dynamics::Matrix, # discrete dynamics of wind W
                    wind_velocity::Vector # current wind state w
                    ) # return the first control command of the solved policy 
    
    # get our sizes for state and control
    nx,nu = size(B)
    
    # check sizes 
    @assert size(A) == (nx, nx)
    @assert length(xic) == nx 
    @assert length(X_ref_window) == N_mpc 
    @assert size(Q) == (nx, nx)
    @assert size(R) == (nu, nu)

    # variables we are solving for
    X = cvx.Variable(nx,N_mpc)
    U = cvx.Variable(nu,N_mpc-1)

    obj = 0 

    for i = 1:N_mpc-1
        obj += (0.5*cvx.quadform(X[:,i]-X_ref_window[i], Q)+0.5*cvx.quadform(U[:,i], R))
    end
    obj += 0.5*cvx.quadform(X[:,N_mpc]-X_ref_window[N_mpc], Q)

    # create problem with objective
    prob = cvx.minimize(obj)

    prob.constraints += (X[:,1] == xic)
    for i = 1:N_mpc-1
        prob.constraints += (X[:,i+1] == A*X[:,i] + B*U[:,i] + wind_dynamics * wind_velocity)
        # prob.constraints += (X[:,i+1] == A*X[:,i] + B*U[:,i])
        prob.constraints += (u_min ≤ U[:,i])
        prob.constraints += (U[:,i] ≤ u_max)
    end
    

    # solve problem 
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = true)

    # get X and U solutions 
    X = X.value
    U = U.value

    # return first control U 
    Ucontrol = U[:,1]
    return Ucontrol, X
end

In [ ]:
# Drone parameters
Ixx= 2.331e2
Iyy= 2.322e2
Izz= 4.022e2
model = (mass= 6.728,
    J= Diagonal([Ixx, Iyy, Izz]), 
    gravity= [0,0,-9.81],
    L= 1.075, 
    kf= 0.1, # constant thrust coefficient 
    km= 0.01, # propeller drag coefficient
    dt= 0.1, # 100 Hz
    α= 0 * (π / 180), # optimal value determined from paper
    β= 0 * (π / 180), # optimal value determined from paper
    λ= (((π/3))) # each propeller is 60 degrees from each other TODO fix this (?)
)
# Simulation parameters
dt = model.dt
N = 300
time = 1:N
n_states = 12
n_inputs = 6

# Initialize Kalman Filter parameters
Qkf = I*1
Rkf = I*1

# Initialize wind parameters
wind_dynamics = zeros(n_states, 3)
wind_dynamics[4, 1] = 1  # Wind effect on vx
wind_dynamics[5, 2] = 1  # Wind effect on vy
wind_dynamics[6, 3] = 1  # Wind effect on vz
wind_velocity = [0, 0, 0] # Intensity of the turbulence
wind_intensity = 0.5

# Generate hover trajectory for hexrotor
X_ref, U_ref = create_ref_hover(model, N+N_mpc, dt, n_inputs)
# Generate equilibrium Point
Xbar = X_ref[1]
Ubar = U_ref[1]

# MPC Parameters
Q = diagm([1,1,1,
    0.1,0.1,0.1,
    0.1,0.1,0.1,
    0.1,0.1,0.1])
R = diagm(ones(n_inputs)*0.005)
N_mpc = 20
u_min = zeros(n_inputs) - Ubar
u_max = 10000*ones(n_inputs)+ Ubar

# check dimensions
@assert length(X_ref[1]) == n_states
@assert length(U_ref[1]) == n_inputs

true_states = []
estimated_states = []
inputs = []

# initialize sim variables
P = I(n_states)
true_state = Xbar + randn(n_states) * 0.01
x_hat = true_state
u_mpc = Ubar
measurements = true_state


# Linearize system
#A, B = linearize(X_ref[1], U_ref[1], dt)
A = FD.jacobian(dx -> rk4(model, hexrotor_dynamics, dx, Ubar, dt), Xbar)
B = FD.jacobian(du -> rk4(model, hexrotor_dynamics, Xbar, du, dt), Ubar)
C = I(n_states)
D = zeros(n_states, n_inputs)

X_mpcs = []

# Update simulation loop to include wind update
for i in time
    # Update wind with turbulence model
    #wind_velocity = update_wind(wind_velocity, dt, wind_intensity)

    # Run KF
    x_hat, P = kalman_filter(A, B, C, D, Qkf, Rkf, x_hat, P, u_mpc, measurements, wind_dynamics, wind_velocity)
    
    push!(estimated_states, x_hat)

    #  given a window of N_mpc timesteps, get current reference trajectory
    X_ref_tilde = X_ref[i:i+N_mpc-1]
    
    # call convex mpc controller with state estimate 
    u_mpc, X_mpc = convex_mpc(A, B, X_ref_tilde, true_state, u_min, u_max, N_mpc, Q, R, wind_dynamics, wind_velocity)
    #u_mpc = u_mpc + U_ref[1]
    # u_mpc = [6000, 0, 0, 0, 0, 0]
    # u_mpc = U_ref[1]
    u_mpc = clamp.(u_mpc+Ubar, u_min, u_max)
    push!(inputs, u_mpc)
    push!(X_mpcs, X_mpc)
    
    # simulate next state
    true_state = rk4(model, hexrotor_dynamics, true_state, u_mpc, dt)
    # true_state = A*(true_state - Xbar) + B*(u_mpc - Ubar) + Xbar
    push!(true_states, true_state)

    measurements = true_state + randn(n_states) * 0.001
end



In [ ]:
# print(X_mpcs)
X_mpc = X_mpcs[1]
print(X_mpc)

for i in 1:n_states
    p = plot(1:N_mpc, X_mpc[i, :], label="MPC trajectory")
    display(p)
end

In [ ]:

# Convert results to arrays for plotting
true_states_plot = hcat(true_states...)
estimated_states_plot = hcat(estimated_states...)
U_ref_plot = hcat(U_ref...)
X_ref_plot = hcat(X_ref...)
inputs_plot = hcat(inputs...)


# Plotting
for i in 1:n_states
    p = plot(time, true_states_plot[i, :], label="True")
    plot!(time, estimated_states_plot[i, :], label="Estimated", linestyle=:dash)
    plot!(time, X_ref_plot[i, 1:N], label = "Reference")
    title!("State $i")
    display(p)
end
for i in 1:n_inputs
    p = plot(time, inputs_plot[i, :], label="MPC Inputs")
    plot!(time, U_ref_plot[i, 1:N], label = "Reference")
    title!("Input $i")
    display(p)
end

In [ ]:
Xsim = hcat()
Xref = hcat()


display(animate_hexrotor(Xsim, Xref, model.dt))